## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [4]:
from sklearn.model_selection import train_test_split
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default

TensorFlow 2.x selected.
2.0.0


In [0]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Collect Data

In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('/content/drive/My Drive/Great_Lakes_Assignments/Lab Internal | Residency 6/prices.csv')


### Check all columns in the dataset

In [0]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

In [0]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'],axis=1,inplace = True)

In [0]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data.shape

(851264, 5)

In [0]:
data_fs = data.iloc[0:1000,]

In [0]:
data_fs.shape

(1000, 5)

In [0]:
data_fs['volume'] = data_fs['volume']/1000000;

In [0]:
data_fs.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


In [0]:
type(data_fs)

pandas.core.frame.DataFrame

In [0]:
dataset = data_fs.values   #Numpy Array 

In [0]:
type(dataset)

numpy.ndarray

In [0]:
# split into input (X) and output (Y) variables
X = dataset[:,0:4]
Y = dataset[:,4]

In [0]:
X.shape

(1000, 4)

In [0]:
Y.shape

(1000,)

### Divide the data into train and test sets

In [0]:
import numpy as np
# fix random seed for reproducibility
np.random.seed(7)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
train_x.dtype

dtype('float64')

In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

In [0]:
train_x.dtype

dtype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)
print('Normalization Done')

Normalization Done


In [0]:
#Reshape the numpy array to column
train_y.shape

(700,)

In [0]:
train_y=train_y.reshape(-1,1)

In [0]:
train_y.shape

(700, 1)

In [0]:
train_x.shape

(700, 4)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
W = tf.Variable(tf.random.normal(shape=[4,1]))
B = tf.zeros(shape=(1))
w1 = tf.Variable(tf.random.normal(shape=[4,3]))
b1 = tf.zeros(shape=(3))

In [0]:
w1

<tf.Variable 'Variable:0' shape=(4, 3) dtype=float32, numpy=
array([[ 1.0143205 , -0.42116386,  0.12676455],
       [ 1.76547   , -0.17084962, -0.14275889],
       [ 0.8056572 ,  0.00948665,  0.15206176],
       [ 0.05490566,  0.75913346, -0.28360245]], dtype=float32)>

In [0]:
b1

<tf.Tensor: id=40126, shape=(3,), dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>

In [0]:
#We are initializing weights and Bias of (Hidden-Output)
w2 = tf.Variable(tf.random.normal(shape=[3,1]))
b2 = tf.zeros(shape=(1))

In [0]:
w2

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[ 0.80350345],
       [-0.3524729 ],
       [-0.10114582]], dtype=float32)>

In [0]:
b2

<tf.Tensor: id=40142, shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>

2.Define a function to calculate prediction

In [0]:
def prediction(x, w1, b1,w2,b2):
    
    xw_matmul = tf.matmul(x, w1)
    net1 = tf.add(xw_matmul, b1)
    y=tf.sigmoid(net1)
    net2=tf.matmul(y, w2)+b2
   #out=tf.sigmoid(net2)
    return net2

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(train_x, train_y, w1, b1,w2,b2, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w1,b1,w2,b2])
        
        net2 = prediction(train_x, w1, b1,w2,b2)
        current_loss =loss(net2, train_y)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw1,db1,dw2,db2 = t.gradient(current_loss,[w1, b1,w2,b2])
    
    #Update Weights at output layer
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    
     #Update Weights at hidden layer
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1
    
    return w1, b1,w2,b2

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [0]:
for i in range(100):
    
    w1, b1,w2,b2 = train(train_x, train_y, w1, b1,w2,b2)
    print("Loss at step {:d}: {:.3f}".format(i, loss(prediction(train_x, w1, b1,w2,b2), train_y)))

Loss at step 0: 243.199
Loss at step 1: 241.014
Loss at step 2: 239.023
Loss at step 3: 237.207
Loss at step 4: 235.548
Loss at step 5: 234.034
Loss at step 6: 232.651
Loss at step 7: 231.386
Loss at step 8: 230.231
Loss at step 9: 229.175
Loss at step 10: 228.211
Loss at step 11: 227.331
Loss at step 12: 226.527
Loss at step 13: 225.794
Loss at step 14: 225.125
Loss at step 15: 224.516
Loss at step 16: 223.960
Loss at step 17: 223.455
Loss at step 18: 222.995
Loss at step 19: 222.578
Loss at step 20: 222.198
Loss at step 21: 221.853
Loss at step 22: 221.540
Loss at step 23: 221.257
Loss at step 24: 221.000
Loss at step 25: 220.767
Loss at step 26: 220.556
Loss at step 27: 220.366
Loss at step 28: 220.193
Loss at step 29: 220.038
Loss at step 30: 219.897
Loss at step 31: 219.770
Loss at step 32: 219.656
Loss at step 33: 219.553
Loss at step 34: 219.460
Loss at step 35: 219.376
Loss at step 36: 219.301
Loss at step 37: 219.233
Loss at step 38: 219.172
Loss at step 39: 219.117
Loss at st

### Get the shapes and values of W and b

In [0]:
w1

<tf.Tensor: id=46533, shape=(4, 3), dtype=float32, numpy=
array([[ 1.179079  , -0.37582338,  0.23125648],
       [ 1.936527  , -0.12113725, -0.0319922 ],
       [ 0.9699632 ,  0.05527784,  0.2567634 ],
       [ 0.22647834,  0.8085627 , -0.17287572]], dtype=float32)>

In [0]:
w1.shape

TensorShape([4, 3])

In [0]:
b1

<tf.Tensor: id=46536, shape=(3,), dtype=float32, numpy=array([0.3359203 , 0.09518117, 0.21541055], dtype=float32)>

In [0]:
b1.shape

TensorShape([3])

In [0]:
w2

<tf.Tensor: id=46527, shape=(3, 1), dtype=float32, numpy=
array([[2.697105 ],
       [0.7606927],
       [0.9810149]], dtype=float32)>

In [0]:
w2.shape

TensorShape([3, 1])

In [0]:
b2

<tf.Tensor: id=46530, shape=(1,), dtype=float32, numpy=array([2.126375], dtype=float32)>

In [0]:
b2.shape

TensorShape([1])

### Model Prediction on 1st Examples in Test Dataset

In [0]:
test_data=test_x[0:1,:]
test_data

array([[0.49863145, 0.49986658, 0.49250004, 0.50886524]], dtype=float32)

In [0]:
pred = prediction(test_data, w1, b1,w2,b2)
pred

<tf.Tensor: id=46557, shape=(1, 1), dtype=float32, numpy=array([[5.626587]], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

In [8]:
import pandas as pd
import numpy as np
print(tf.__version__)

2.0.0


In [0]:
#setting random seed

tf.random.set_seed(42)

### Load the given Iris data using pandas (Iris.csv)

In [0]:
data_iris = pd.read_csv('/content/drive/My Drive/Great_Lakes_Assignments/Lab Internal | Residency 6/11_Iris.csv')

In [11]:
data_iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
y = pd.get_dummies(data_iris['Species'])

In [13]:
y.head()

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [0]:
data_iris.drop(['Id'],axis=1,inplace=True);

In [15]:
data_iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [16]:
x = data_iris.iloc[:,0:4]
x.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [17]:
x.shape

(150, 4)

In [18]:
y.shape

(150, 3)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=.30,random_state=2)

In [20]:
train_x.dtypes

SepalLengthCm    float64
SepalWidthCm     float64
PetalLengthCm    float64
PetalWidthCm     float64
dtype: object

In [21]:
import numpy as np
from pandas import read_csv
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

In [23]:
train_x.dtype

dtype('float32')

In [24]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

2.0.0


In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(4, input_dim = 4, kernel_initializer='normal', activation='sigmoid'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))    
    # Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics = ['accuracy'])


### Model Training 

In [26]:
output = model.fit(train_x, train_y,validation_data = (test_x,test_y), epochs = 50)
print(output)

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 1s 13ms/sample - loss: 1.2317 - accuracy: 0.3333 - val_loss: 1.1639 - val_accuracy: 0.3333
Epoch 2/50
105/105 [==============================] - 0s 305us/sample - loss: 1.2027 - accuracy: 0.3333 - val_loss: 1.1485 - val_accuracy: 0.3333
Epoch 3/50
105/105 [==============================] - 0s 283us/sample - loss: 1.1866 - accuracy: 0.3333 - val_loss: 1.1360 - val_accuracy: 0.3333
Epoch 4/50
105/105 [==============================] - 0s 252us/sample - loss: 1.1732 - accuracy: 0.3333 - val_loss: 1.1226 - val_accuracy: 0.3333
Epoch 5/50
105/105 [==============================] - 0s 244us/sample - loss: 1.1583 - accuracy: 0.3333 - val_loss: 1.1119 - val_accuracy: 0.3333
Epoch 6/50
105/105 [==============================] - 0s 228us/sample - loss: 1.1468 - accuracy: 0.3333 - val_loss: 1.1071 - val_accuracy: 0.3333
Epoch 7/50
105/105 [==============================] - 0s 235us/sample - loss: 1.

### Model Prediction

In [27]:
y_pred = model.predict(test_x)
y_pred

array([[0.3971141 , 0.32020667, 0.28267923],
       [0.3921645 , 0.32653087, 0.28130466],
       [0.30324328, 0.3937308 , 0.30302587],
       [0.39474145, 0.32387978, 0.28137875],
       [0.38754308, 0.32588324, 0.28657368],
       [0.30597362, 0.3766951 , 0.3173313 ],
       [0.3891594 , 0.32633257, 0.28450805],
       [0.29783902, 0.39134404, 0.31081697],
       [0.30020267, 0.38872224, 0.31107512],
       [0.39199308, 0.32573113, 0.28227577],
       [0.40083143, 0.30961752, 0.28955102],
       [0.39779466, 0.3231206 , 0.27908468],
       [0.40085065, 0.31594002, 0.28320932],
       [0.3962309 , 0.31051797, 0.2932511 ],
       [0.32485664, 0.36550388, 0.3096395 ],
       [0.3190946 , 0.37142488, 0.3094805 ],
       [0.3840054 , 0.34039524, 0.27559933],
       [0.32603297, 0.37106365, 0.30290335],
       [0.30211738, 0.38063768, 0.31724492],
       [0.3240374 , 0.3776745 , 0.2982881 ],
       [0.31430823, 0.37248713, 0.31320465],
       [0.3263261 , 0.3653242 , 0.3083497 ],
       [0.

In [30]:
y_pred.shape

(45, 3)

### Save the Model

In [0]:
from keras.models import model_from_json

In [33]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [35]:
model.save('iris_model.html')
print("Saved model to disk");

INFO:tensorflow:Assets written to: iris_model.html/assets
Saved model to disk


### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?